## Notebook setup

In [ ]:
import scanpy as sc
import scanpy.external as sce
import numpy as np
import pandas as pd
import warnings, scipy.sparse as sp, matplotlib, matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import rc_context
from collections import Counter
import matplotlib.font_manager
import pyreadr
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
import magic
#import seaborn as sns
import palantir
import loompy
#from scipy.sparse import csgraph

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rc('font', size=14)

pd.set_option('display.max_rows', 200)

sc.set_figure_params(dpi=80, dpi_save=300, color_map='Spectral_r', vector_friendly=True, transparent=True)
sc.settings.verbosity = 0 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

In [ ]:
# preset color palettes and color maps
user_defined_palette =  [ '#F6222E', '#16FF32', '#3283FE', '#FEAF16', '#BDCDFF', '#3B00FB', '#1CFFCE', '#C075A6', '#F8A19F', '#B5EFB5', '#FBE426', '#C4451C', 
                          '#2ED9FF', '#c1c119', '#8b0000', '#FE00FA', '#1CBE4F', '#1C8356', '#0e452b', '#AA0DFE', '#B5EFB5', '#325A9B', '#90AD1C']

user_defined_cmap_markers = LinearSegmentedColormap.from_list('mycmap', ["#E6E6FF", "#CCCCFF", "#B2B2FF", "#9999FF",  "#6666FF",   "#3333FF", "#0000FF"])
user_defined_cmap_degs = LinearSegmentedColormap.from_list('mycmap', ["#0000FF", "#3333FF", "#6666FF", "#9999FF", "#B2B2FF", "#CCCCFF", "#E6E6FF", "#E6FFE6", "#CCFFCC", "#B2FFB2", "#99FF99", "#66FF66", "#33FF33", "#00FF00"])

In [ ]:
# display plot directly below code cell
%matplotlib inline

In [ ]:
def observe_variance(anndata_object):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    # variance per principal component
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = anndata_object.uns['pca']['variance_ratio']
    ax1.scatter(x,y,s=4)
    ax1.set_xlabel('PC')
    ax1.set_ylabel('Fraction of variance explained\n')
    ax1.set_title('Fraction of variance explained per PC\n')
    # cumulative variance explained
    cml_var_explained = np.cumsum(anndata_object.uns['pca']['variance_ratio'])
    x = range(len(anndata_object.uns['pca']['variance_ratio']))
    y = cml_var_explained
    ax2.scatter(x,y,s=4)
    ax2.set_xlabel('PC')
    ax2.set_ylabel('Cumulative fraction of variance\nexplained')
    ax2.set_title('Cumulative fraction of variance\nexplained by PCs')
    fig.tight_layout()
    plot = plt.show
    return(plot)

In [ ]:
def label_transfer(dist, labels):
    lab = pd.get_dummies(labels).to_numpy().T
    class_prob = lab @ dist
    norm = np.linalg.norm(class_prob, 2, axis=0)
    class_prob = class_prob / norm
    class_prob = (class_prob.T - class_prob.min(1)) / class_prob.ptp(1)
    return class_prob

## Perform quality control and clean-up samples

### Load spaceranger output files

In [ ]:
from glob import glob

counter=0
bad_clusters_per_sample = [['4', '8'],  ['4', '5', '6'], ['7', '8'], ['1', '10'], ['2', '3', '6', '7'], ['2', '5', '9'], ['2', '7'], ['0', '1', '6', '8', '10', '11']]
visiums_list = []

h5_path = glob("../data/internal/10xGenomics/Visium/LJ184/LJ184_S_0*/sr-results/", recursive = True)

#h5_path = ['../data/10xVisium/LJ184/LJ184_S_003/sr-results/',
# '../data/10xVisium/LJ184/LJ184_S_019/sr-results/']

for path in h5_path: 
    print(path)
    tmp_visium = sc.read_visium(path=path, count_file="filtered_feature_bc_matrix.h5", source_image_path=path)
    tmp_visium.var_names_make_unique()
    tmp_visium.raw = tmp_visium # keep a copy of the raw adata 
    
    np.random.seed(42) 
    index_list = np.arange(tmp_visium.shape[0]) # randomize the order of cells for plotting
    np.random.shuffle(index_list)
    tmp_visium = tmp_visium[index_list]
    
    sc.pp.calculate_qc_metrics(tmp_visium, inplace=True)
    tmp_visium.obs['original_total_counts'] = tmp_visium.obs['total_counts'] # store unfiltered/unprocessed data
    tmp_visium.obs['log10_original_total_counts'] = np.log10(tmp_visium.obs['original_total_counts']) # store unfiltered/unprocessed data 
 
    tmp_visium.var['mt'] = tmp_visium.var_names.str.startswith(('MT-', 'mt-')) # mitochondrial genes
    tmp_visium.var['ribo'] = tmp_visium.var_names.str.startswith(('RPS','RPL', 'Rps', 'Rpl')) # ribosomal genes    
    tmp_visium.var['hb'] = tmp_visium.var_names.str.startswith(('^Hb', '^HB')) # hemoglobin genes

    tmp_visium.obs['mito_frac'] = np.sum(tmp_visium[:,tmp_visium.var['mt']==True].X, axis=1) / np.sum(tmp_visium.X, axis=1) # compute fraction of mitochondrial genes vs all genes per cell
    tmp_visium.obs['ribo_frac'] = np.sum(tmp_visium[:,tmp_visium.var['ribo']==True].X, axis=1) / np.sum(tmp_visium.X, axis=1) # compute fraction of ribosomal genes vs all genes per cell
    tmp_visium.obs['hb_frac'] = np.sum(tmp_visium[:,tmp_visium.var['hb']==True].X, axis=1) / np.sum(tmp_visium.X, axis=1) # compute fraction of hemoglobin genes vs all genes per cell

    sc.pp.filter_genes(tmp_visium, min_cells=1) # remove genes that are not expressed in any cells
    print(tmp_visium.shape)
    tmp_visium = tmp_visium[:,tmp_visium.var['ribo']==False] # remove ribosomal genes
    print(tmp_visium.shape)
    tmp_visium = tmp_visium[:,tmp_visium.var['hb']==False] # remove hemoglobin genes
    print(tmp_visium.shape)

    sc.pp.normalize_per_cell(tmp_visium, counts_per_cell_after=10**4)
    sc.pp.log1p(tmp_visium)

    sc.tl.pca(tmp_visium, n_comps=200, svd_solver='arpack', random_state=np.random.RandomState(42), use_highly_variable=False)
    observe_variance(tmp_visium)
    sc.tl.pca(tmp_visium, n_comps=30,  svd_solver='arpack', random_state=np.random.RandomState(42), use_highly_variable=False)
    sc.pp.neighbors(tmp_visium, n_neighbors=15, random_state=42)
    sc.tl.umap(tmp_visium)
    sc.pl.umap(tmp_visium, 
        color=['log10_original_total_counts', 'n_genes_by_counts','ribo_frac', 'mito_frac'], 
        palette='tab20',  
        color_map='Spectral_r', 
        ncols=4,
        wspace = 0.5,
        sort_order = False
    )
    
    for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        sc.tl.leiden(tmp_visium, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))
    sc.pl.umap(tmp_visium, 
    color=['leiden_0.1', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.5', 
           'leiden_0.6', 'leiden_0.7', 'leiden_0.8', 'leiden_0.9', 'leiden_1.0'], 
    palette=user_defined_palette,  
    color_map='Spectral_r', 
    use_raw=False,
    ncols=5,
    wspace = 0.7,
    outline_width=[0.6, 0.05],
    frameon=False,
    add_outline=True,
    sort_order = False
    )

    sc.pl.spatial(tmp_visium, 
              img_key="hires",
              color=['log10_original_total_counts', 'n_genes_by_counts','ribo_frac', 'mito_frac'], 
              size=1.5, 
              color_map='Spectral_r', 
              ncols=4)
    sc.pl.spatial(tmp_visium, 
              img_key="hires", 
              size=1.5, 
              color_map='Spectral_r', 
              ncols=4)
    
    sc.pl.spatial(tmp_visium, 
              img_key="hires", 
              color = ['leiden_1.0'], 
              size=1.5, 
              color_map='Spectral_r', 
              ncols=4)
    
    clusters_to_remove = bad_clusters_per_sample[counter]
    cluster_filter = [x not in clusters_to_remove for x in tmp_visium.obs['leiden_1.0']]
    print('Total number of cells pre-filtering: ' + str(tmp_visium.shape[0]))
    print('Number of cells to keep after filtering: ' + str(sum(cluster_filter)))
    tmp_visium_filtered = tmp_visium[cluster_filter]
    tmp_visium = tmp_visium_filtered
    
    sc.pp.highly_variable_genes(tmp_visium, n_top_genes=5000, n_bins=20, flavor='seurat',  inplace=True)   
    sc.tl.pca(tmp_visium, n_comps=200, svd_solver='arpack', random_state=np.random.RandomState(42), use_highly_variable=True)
    observe_variance(tmp_visium)
    sc.tl.pca(tmp_visium, n_comps=30, svd_solver='arpack', random_state=np.random.RandomState(42), use_highly_variable=True)
    sc.pp.neighbors(tmp_visium, n_neighbors=15)
    sc.tl.umap(tmp_visium)
    sc.pl.umap(tmp_visium, 
        color=['log10_original_total_counts', 'n_genes_by_counts','ribo_frac', 'mito_frac'], 
        palette='tab20',  
        color_map='Spectral_r', 
        ncols=4,
        wspace = 0.5,
        sort_order = False
    )
    
    for resolution_parameter in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        sc.tl.leiden(tmp_visium, resolution=resolution_parameter, random_state=42, 
                        key_added='leiden_'+str(resolution_parameter))    
        
    tmp_visium.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
    sc.pl.umap(tmp_visium, 
        color=['leiden_0.2'], 
        color_map='Spectral_r', 
        ncols=4,
        wspace = 0.5,
        sort_order = False
    )
    
    sc.pl.spatial(tmp_visium, 
              img_key="hires", 
              size=1.5, 
              color_map='Spectral_r', 
              ncols=4)

    sc.pl.spatial(tmp_visium, 
                  img_key="hires", 
                  color = ['leiden_0.2'], 
                  palette=['#7fc97f', '#f0027f'],
                  color_map='Spectral_r', 
                  size=1.5, 
                  ncols=4)
    #path_to_h5ad = '../output/metadata/10x_visium_02mo_day0.h5ad'
    #adata.write(path_to_h5ad)
    visiums_list.append(tmp_visium)
    counter = counter + 1

## Combine in a massive spatial dataset

In [ ]:
#spatial = sc.concat(
 #   visiums_list,
  #  label="library_id",
  #  uns_merge="unique",
  #  keys=[
       # k
      #  for d in [
         #   visiums_list[0].uns["spatial"],
          #  visiums_list[1].uns["spatial"],
           # visiums_list[2].uns["spatial"],
            #visiums_list[3].uns["spatial"],
            #visiums_list[4].uns["spatial"],
            #visiums_list[5].uns["spatial"],
            #visiums_list[6].uns["spatial"],
           # visiums_list[7].uns["spatial"]
        #]
      #  for k, v in d.items()
   # ],
   # index_unique="-",
#)

## Scanorama spatial-RNAseq integration

In [ ]:
import scanorama
import numpy as np

### (same as for Figure 3)

### Steady state

In [ ]:
path_to_h5ad = '../output/metadata/anndata_objects/main_fig1_annotated.h5ad'

In [ ]:
adata_d0 = sc.read_h5ad(path_to_h5ad)
adata_d0.uns['log1p']["base"] = None

In [ ]:
adata_d0_18mo = adata_d0[adata_d0.obs['stage']=='18mo']

In [ ]:
adata_d0_18mo.shape

In [ ]:
adata_d0_02mo = adata_d0[adata_d0.obs['stage']=='02mo']

In [ ]:
adata_d0_02mo.shape

### Damage

In [ ]:
path_to_h5ad = '../output/metadata/anndata_objects/ext_fig5_annotated.h5ad'

In [ ]:
adata_d147 = sc.read_h5ad(path_to_h5ad)
adata_d147.uns['log1p']["base"] = None

In [ ]:
adata_d1_18mo = adata_d147[(adata_d147.obs['stage']=='18mo') & (adata_d147.obs['day']=='d1')]

In [ ]:
adata_d1_18mo.shape

In [ ]:
adata_d1_02mo = adata_d147[(adata_d147.obs['stage']=='02mo') & (adata_d147.obs['day']=='d1')]

In [ ]:
adata_d1_02mo.shape

In [ ]:
adata_d4_18mo = adata_d147[(adata_d147.obs['stage']=='18mo') & (adata_d147.obs['day']=='d4')]

In [ ]:
adata_d4_18mo.shape

In [ ]:
adata_d4_02mo = adata_d147[(adata_d147.obs['stage']=='02mo') & (adata_d147.obs['day']=='d4')]

In [ ]:
adata_d4_02mo.shape

In [ ]:
adata_d7_18mo = adata_d147[(adata_d147.obs['stage']=='18mo') & (adata_d147.obs['day']=='d7')]

In [ ]:
adata_d7_18mo.shape

In [ ]:
adata_d7_02mo = adata_d147[(adata_d147.obs['stage']=='02mo') & (adata_d147.obs['day']=='d7')]

In [ ]:
adata_d7_02mo.shape

### Combined

In [ ]:
path_to_h5ad = '../output/metadata/anndata_objects/main_fig3.h5ad'

In [ ]:
adata_d0147 = sc.read_h5ad(path_to_h5ad)
adata_d0147.uns['log1p']["base"] = None

### Transfer annotation

In [ ]:
annotated_subsets = pd.concat([adata_d0.obs['cell_type_subset'], adata_d147.obs['cell_type_subset']])

In [ ]:
adata_d0147.obs['cell_type_subset']=''

In [ ]:
adata_d0147.obs['cell_type_subset'][adata_d0147.obs.index.isin(annotated_subsets.index) == True] = annotated_subsets

In [ ]:
subset_palette =  ['#F6222E', '#3283FE', '#16FF32', '#BDCDFF', '#3B00FB', '#1CFFCE', '#d62728', '#19c9b3','#FFA5D2',   'grey', '#2ED9FF', '#c1c119', '#8b0000', '#FE00FA', "#F8A19F", '#1CBE4F','#B5EFB5',  '#AA0DFE','#FEAF16', '#325A9B', '#C075A6', 'black']

sc.pl.umap(adata_d0147, color=[ 'cell_type_subset'], 
                        color_map='Spectral_r',
                        palette=subset_palette,
                        use_raw=False, 
                        ncols=4, 
                        wspace = 0.3,
                        outline_width=[0.6, 0.05], 
                        size=15,  
                        frameon=False, 
                        add_outline=True, 
                        sort_order = False)

In [ ]:
h5_path

In [ ]:
# List of datasets:
adatas = [adata_d0_18mo, visiums_list[3]]

# Integration. 
scanorama.integrate_scanpy(adatas)

# Batch correction.
#corrected = scanorama.correct_scanpy(adatas)

# Integration and batch correction.
#corrected = scanorama.correct_scanpy(adatas, return_dimred=True)

In [ ]:
adatas[0].obsm['X_scanorama'].shape

In [ ]:
total = adata_d0_18mo.concatenate(
    visiums_list[3],
    batch_key="dataset",
    batch_categories=["10x_Chromium", "10x_Visium"],
    join="outer",
    uns_merge="first",
)

In [ ]:
del(total)

In [ ]:
# Get all the integrated matrices.
scanorama_int = [ad.obsm['X_scanorama'] for ad in adatas]

# make into one matrix.
all_s = np.concatenate(scanorama_int)
print(all_s.shape)

# add to the AnnData object
total.obsm["scanorama_embedding"] = all_s

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

distances = 1 - cosine_distances(
    total[total.obs.dataset == "10x_Chromium"].obsm[
        "scanorama_embedding"
    ],
    total[total.obs.dataset == "10x_Visium"].obsm[
        "scanorama_embedding"
    ],
)

In [ ]:
class_prob = label_transfer(distances, adata_d0_18mo.obs.cell_type_subset)

In [ ]:
cp_df = pd.DataFrame(
    class_prob, columns=np.sort(adata_d0_18mo.obs.cell_type_subset.unique())
)

cp_df.index =  visiums_list[3].obs.index

In [ ]:
adata_transfer =  visiums_list[3].copy()
adata_transfer.obs = pd.concat(
    [ visiums_list[3].obs, cp_df], axis=1
)

In [ ]:
subset_palette =  ['#F6222E', '#3283FE', '#16FF32', '#BDCDFF', '#3B00FB', '#1CFFCE', '#d62728', '#19c9b3','#FFA5D2',   'grey', '#2ED9FF', '#c1c119', '#8b0000', '#FE00FA', "#F8A19F", '#1CBE4F','#B5EFB5',  '#AA0DFE','#FEAF16', '#325A9B', '#C075A6', 'black']

sc.pl.umap(adata_d4_18mo, color=[ 'cell_type_subset'], 
                        color_map='Spectral_r',
                        palette=subset_palette,
                        use_raw=False, 
                        ncols=4, 
                        wspace = 0.3,
                        outline_width=[0.6, 0.05], 
                        size=15,  
                        frameon=False, 
                        add_outline=True, 
                        sort_order = False)

In [ ]:
### Across cell type signatures

In [ ]:
artEC_sign = ['Tm4sf1',	'Cldn5',	'Ly6c1',	'Ptprb',	'Pecam1',	'Egfl7',	'Ptprm',	'Flt1',	'Cav1',	'Epas1',	'Mecom',	'Fabp4',	'Slc9a3r2',	'Icam2',	'Esam',	'Sox17',	'Cdh5',	'Fbln5',	'Id1',	'Eng',	'Cyyr1',	'Ecscr',	'Adgrf5',	'Arhgap31',	'Cst3',	'Ly6e',	'Podxl',	'Plcb1',	'Pdgfd',	'Cd36',	'Cd200',	'Calm1',	'Stmn2',	'Prex2',	'Klf2',	'Serinc3',	'Mmrn2',	'Sema3g',	'S1pr1',	'Tspan13',	'Cdh13',	'Tinagl1',	'Ldb2',	'Arl15',	'Cav2',	'Crip2',	'Gnai2',	'Cavin2',	'Atox1',	'Hspb1']
capEC_sign = ['Fabp4',	'Gpihbp1',	'Cd36',	'Ly6c1',	'Rgcc',	'Egfl7',	'Tmsb4x',	'Flt1',	'Mgll',	'Pecam1',	'Cav1',	'Cd300lg',	'Cavin2',	'Cdh5',	'Gng11',	'Tspan13',	'Esam',	'Hspb1',	'Emcn',	'Adgrl4',	'Cldn5',	'Tcf15',	'Calm1',	'Ptprm',	'Cyyr1',	'Cd200',	'Tm4sf1',	'Ptprb',	'Etl4',	'Kdr',	'Cav2',	'Arhgap31',	'Kank3',	'Lims2',	'Ctla2a',	'Scarb1',	'Ushbp1',	'Gimap6',	'C1qtnf9',	'Eng',	'Slc9a3r2',	'Car4',	'Fli1',	'Id1',	'Adgrf5',	'Cdh13',	'Aqp1',	'Serinc3',	'Nrp1',	'Ablim3']
venEC_sign = ['Aqp1',	'Pecam1',	'Plvap',	'Ctla2a',	'Egfl7',	'Eng',	'Fabp4',	'Lrg1',	'Ptprb',	'Flt1',	'Tspan7',	'Tmsb4x',	'Mmrn2',	'Vwf',	'Emcn',	'Cdh5',	'Mecom',	'Il6st',	'Ldb2',	'Esam',	'Fli1',	'Gnai2',	'Scarb1',	'Gng11',	'Prkch',	'Ecscr',	'Ablim1',	'Cyyr1',	'Epas1',	'Adgrf5',	'Gimap6',	'Myh9',	'Grrp1',	'Cav1',	'Selp',	'Ptprm',	'Ehd4',	'Fry',	'Abcg2',	'Adgrl4',	'Arhgap31',	'Pdlim1',	'Cd93',	'Tm4sf1',	'2200002D01Rik',	'Ece1',	'St6galnac3',	'Ndrg1',	'Cd200',	'Cd36']
capsFB_sign = ['Pi16',	'Timp2',	'Fn1',	'Mfap5',	'Clec3b',	'Cd248',	'Opcml',	'Nid1',	'Pcolce2',	'Igfbp6',	'Fbn1',	'Tmem100',	'Fndc1',	'Ackr3',	'Fstl1',	'Ebf2',	'Creb5',	'Col14a1',	'Sema3c',	'Anxa3',	'Pla1a',	'Adgrd1',	'Pcsk6',	'Smpd3',	'Dpt',	'Ddr2',	'Axl',	'Tnxb',	'Ogn',	'Loxl1',	'Dpp4',	'Adamts5',	'Islr',	'Plpp3',	'Scara5',	'Ly6c1',	'Lsp1',	'Efhd1',	'Cd34',	'Gfpt2',	'Tppp3',	'Col1a1',	'Sdk1',	'Col3a1',	'Rnase4',	'Emilin2',	'Metrnl',	'Sparc',	'Col1a2',	'Tgfbr2']
intFB_sign = ['Gsn',	'Dcn',	'Serpinf1',	'Smoc2',	'Lrp1',	'Lum',	'Htra3',	'Pcolce',	'Col3a1',	'Gpx3',	'Col1a2',	'Abca8a',	'Lpl',	'Celf2',	'Cygb',	'Fbln1',	'Selenop',	'Slit3',	'Gas1',	'Col1a1',	'Mmp2',	'Dpep1',	'Penk',	'Col15a1',	'Inmt',	'Serpinh1',	'Clec3b',	'Rbp1',	'Igfbp4',	'Dpt',	'Pdgfra',	'Mgst1',	'Igf1',	'Ftl1',	'Lama2',	'Serping1',	'Rnase4',	'Il11ra1',	'Mfap5',	'Ogn',	'Tmsb10',	'Cd302',	'Svep1',	'Aebp1',	'Bgn',	'Plxdc2',	'Nbl1',	'Ifitm2',	'Itm2a',	'Adamts12']
medFB_sign = ['Serpine2',	'Bgn',	'Csmd1',	'Enpp2',	'Ptn',	'Apod',	'Igfbp7',	'Lhfp',	'Lsamp',	'Hsd11b1',	'Ifitm1',	'Ltc4s',	'Ccl19',	'Cd63',	'Colec12',	'Des',	'Col15a1',	'Mgp',	'Ank2',	'Sparcl1',	'Cp',	'Spon1',	'Gpm6b',	'Il34',	'Tmem176a',	'Tmem176b',	'Apoe',	'C3',	'Ltbp1',	'Lama2',	'Cygb',	'Tcf4',	'Ndufa4l2',	'Cd9',	'Lum',	'Col6a2',	'Abi3bp',	'Crispld2',	'Dcn',	'Laptm4a',	'Ifitm2',	'G0s2',	'Jun',	'Mfge8',	'Cd302',	'Il1r1',	'Nrp1',	'Pde7b',	'Prelp',	'Cd81']
MEC_sign = ['Gpm6a',	'Nkain4',	'Upk3b',	'Clu',	'Gm12840',	'Igfbp6',	'Abi1',	'Aebp1',	'Trf',	'Crip1',	'Krt19',	'Igfbp5',	'Rspo1',	'Wt1',	'Mpp6',	'Upk1b',	'Rarres2',	'Msln',	'Il1rapl1',	'Plxna4',	'C2',	'Efemp1',	'Adamtsl1',	'Sntg1',	'Pkhd1l1',	'Sox6',	'Aldh1a2',	'C4b',	'Gpc3',	'Kcnd2',	'Lgals7',	'Gas6',	'Cldn15',	'Lrrn4',	'Mgp',	'Hspb1',	'Csrp2',	'Cav1',	'Bicd1',	'Tmem151a',	'Wdr17',	'Cldn10',	'Cavin2',	'Efna5',	'Ptgis',	'Flrt2',	'Ezr',	'1010001N08Rik',	'C3',	'Muc16']
PCvSMC_sign = ['Myl9',	'Cald1',	'Gm13889',	'Tpm2',	'Acta2',	'Prkg1',	'Tagln',	'Tpm1',	'Rgs5',	'Ctnna3',	'Tinagl1',	'Cacna1c',	'Crip1',	'Myh11',	'Notch3',	'Mylk',	'Dmd',	'Myl6',	'Sncg',	'Mustn1',	'Ndufa4l2',	'Sparcl1',	'Dgkb',	'Gng11',	'Kcnab1',	'Ppp1r12a',	'Higd1b',	'Des',	'Pde3a',	'Pcp4l1',	'Rcan2',	'Bcam',	'Calm2',	'Gucy1a1',	'Mef2c',	'Ptp4a3',	'Aspn',	'Cacnb2',	'Cpe',	'Rgs4',	'Igfbp7',	'Csrp1',	'Gucy1b1',	'Lmod1',	'Cox4i2',	'Mfge8',	'Timp3',	'Tm4sf1',	'Ppp1r14a',	'Malat1']
nmSC_sign = ['Csmd1',	'Kcna1',	'Fxyd1',	'Plp1',	'Dlgap1',	'Prnp',	'Cd9',	'Gfra3',	'Stard13',	'Vwa1',	'Cdh19',	'Malat1',	'Dbi',	'Slc35f1',	'Aspa',	'Scn7a',	'Cd59a',	'Zeb2',	'S100b',	'Cadm2',	'Matn2',	'Nkain2',	'Cryab',	'Gpm6b',	'Gpr37l1',	'Art3',	'Apoe',	'Cnp',	'Vim',	'Sox10',	'Zfp536',	'Lgi4',	'Nrn1',	'Lgals3',	'Sgcd',	'Fcgr2b',	'Ncam1',	'Frmd4a',	'Sat1',	'Chl1',	'Cadm1',	'S100a6',	'Prkca',	'Gatm',	'Fign',	'Pmp22',	'Kcna2',	'Sfrp5',	'Fgl2',	'Csrp1']
fat_sign = ['Mgst1',	'Lpl',	'Dbi',	'Apoe',	'Hp',	'Car3',	'Aoc3',	'Mpc2',	'Ndufa4',	'G0s2',	'Cebpa',	'Fabp4',	'Rarres2',	'Rbp4',	'Plin2',	'Vim',	'mt-Atp6',	'mt-Co1',	'mt-Co3',	'mt-Nd1',	'Angptl4',	'Pparg',	'Eif4ebp1',	'Adam12',	'Adipoq',	'Ndufb9',	'Cox7b',	'Mmd',	'Etfb',	'Nnmt',	'Slc1a5',	'Slc24a3',	'Gsn',	'Selenbp1',	'Fcor',	'mt-Nd4',	'Uqcr11',	'Steap4',	'Adhfe1',	'mt-Cytb',	'Arhgap24',	'Prkar2b',	'Chchd2',	'Chpt1',	'Plin1',	'Atp5g3',	'Hadh',	'Zeb2',	'Aldh2',	'Col15a1']
aaTEC1_sign = ['Krt18',	'Epcam',	'Trpm3',	'Cd24a',	'Wfdc18',	'Krt8',	'Spint2',	'Ifi27l2a',	'Meis2',	'Cldn3',	'Ly6e',	'Slc16a11',	'Slc9a3r1',	'S100a11',	'Btg1',	'Perp',	'Gsta4',	'Isl1',	'Fxyd3',	'Pbx1',	'Cdk19',	'Ptprd',	'mt-Co1',	'Taldo1',	'BC006965',	'Apobec3',	'S100a1',	'Isg15',	'Pde4b',	'Oasl2',	'Anxa2',	'Aldoc',	'Eya1',	'Socs2',	'Stat1',	'Sdc4',	'Rtp4',	'2610307P16Rik',	'Fam107a',	'Mif',	'mt-Co3',	'Cdh1',	'Cd74',	'Atp1a1',	'Fbxo2',	'Nedd4l',	'Shank2',	'Eno1',	'Marcksl1',	'mt-Nd1']
aaTEC2_sign = ['Csmd1',	'Ndrg2',	'Ccl19',	'Trpm3',	'Cd74',	'Nav2',	'Gadd45g',	'H2-Ab1',	'Atp1b1',	'Pde4b',	'Csrp1',	'H2-Eb1',	'Btg1',	'Meis2',	'Mir100hg',	'Ccl21a',	'Socs2',	'Iigp1',	'Tagln',	'Socs3',	'H2-Aa',	'Kirrel3',	'Ptprd',	'Notch3',	'Tagln2',	'Heyl',	'Gucy1a1',	'Bcl2',	'Gm48742',	'Gpx3',	'Eya4',	'Cnn3',	'Ncam1',	'Art3',	'Cd9',	'Eya1',	'Apoe',	'Pbx1',	'Ar',	'Nhs',	'Olfm2',	'Cul1',	'Fos',	'Palld',	'Phlda1',	'Jmjd1c',	'Sncaip',	'Cacna1c',	'Sdc4',	'Arid5b']
cTEC_sign = ['Krt18',	'Cstb',	'Krt8',	'Ctsl',	'Ank3',	'Ndufa11',	'Tbata',	'H2-Ab1',	'Slc46a2',	'Prxl2b',	'H2-Aa',	'Prss16',	'Cd74',	'Gas6',	'Fabp5',	'Nlgn1',	'Pax1',	'H2-Eb1',	'Pltp',	'Ccl25',	'Wnt4',	'Tmem131l',	'Rbfox1',	'Psmb9',	'Dpp6',	'Shisa2',	'AI646519',	'Psmb11',	'H2-DMa',	'Bnip3l',	'Krt5',	'Sfn',	'H2-DMb2',	'Tsc22d1',	'Snhg11',	'Krt17',	'Trp63',	'Ctnnd2',	'Nav2',	'Atpif1',	'Kctd1',	'Ccl21a',	'Sh2d4b',	'Perp',	'Tenm4',	'Dsp',	'Spint2',	'Ociad2',	'Ndrg3',	'Apobec3']
mTEC1_sign = ['H2-Ab1',	'Krt5',	'H2-Eb1',	'H2-Aa',	'Ifi27l2a',	'Ank3',	'Cd74',	'Ccl21a',	'Krt14',	'Ifitm3',	'Krt18',	'Perp',	'Fxyd3',	'Eya4',	'Epcam',	'Spint2',	'Isg15',	'Oasl2',	'Sfn',	'Meis2',	'Mif',	'Krt17',	'H2-DMa',	'Atp1a1',	'Rbfox1',	'Ly6e',	'Krt8',	'Nxn',	'Ctnnd2',	'Rtp4',	'Kcnma1',	'H3f3b',	'Apobec3',	'Ptprd',	'Pbx1',	'Apoe',	'Atp1b1',	'Trp63',	'Atpif1',	'Gas6',	'Eya1',	'Urah',	'Itga6',	'Dsp',	'H2-DMb1',	'Marcksl1',	'Csrp1',	'Fcgbp',	'B2m',	'Cdh1']
mTECprol_sign = ['Krt17',	'Tpm2',	'Ascl1',	'Ccl21a',	'Ptma',	'Krt7',	'Spint2',	'Krt5',	'Cd74',	'Marcksl1',	'H3f3b',	'H2-Eb1',	'Wfdc18',	'Sox4',	'H2-Aa',	'H2-Ab1',	'Hes6',	'Epcam',	'Ubd',	'Skint10',	'Krt8',	'Krt14',	'Tubb2b',	'Pfn1',	'Actg1',	'Prxl2b',	'Fcgbp',	'Mapk13',	'H2afy2',	'Rgs5',	'Mif',	'Cd82',	'Mdk',	'H2-DMb2',	'Gm49708',	'Kcnq3',	'Ppia',	'Srgn',	'Cdk4',	'Adm',	'Hagh',	'Fezf2',	'Hsp90ab1',	'S100a14',	'Hnrnpa1',	'Cib1',	'H2-Q7',	'Ank3',	'Perp',	'Slc25a5']
mTEC2_sign = ['Cd74',	'Srgn',	'Ubd',	'H2-Aa',	'H2-Eb1',	'Syt1',	'S100a14',	'H2-Ab1',	'H2-DMb2',	'Cyba',	'H2-Oa',	'Mrpl38',	'Lrrc42',	'Marcksl1',	'Hagh',	'Fabp5',	'Cd52',	'Fezf2',	'Aire',	'Cib1',	'Bspry',	'Krt17',	'Hspb11',	'Plb1',	'Krt8',	'Fcgbp',	'Cdx1',	'Psme2',	'Dpp10',	'Dio1',	'Spint2',	'Ctss',	'H2-Eb2',	'Skint10',	'Txn1',	'Ankrd33b',	'Tnfrsf11a',	'Fam89a',	'Syngr2',	'Calcb',	'Nfkbia',	'Ndufc2',	'Laptm5',	'Utf1',	'Cox17',	'Il4i1',	'Hdc',	'Ing1',	'Gm47938',	'Csn2']
mTEC3_sign = ['Ly6d',	'Fxyd3',	'Sfn',	'Perp',	'Dapl1',	'Krt17',	'Dsp',	'Tacstd2',	'Dmkn',	'Skint3',	'Spink5',	'H2afj',	'Dstn',	'Epcam',	'H2-K1',	'Oit1',	'Cdh1',	'Cst6',	'Gsta4',	'Hspb1',	'Fabp5',	'Pdzk1ip1',	'Cdkn2b',	'Urah',	'S100a14',	'Cdkn2a',	'H2-Q7',	'Spint2',	'Bcl2a1b',	'Cldn4',	'Atox1',	'Vamp8',	'Calml3',	'Lypd3',	'Avpi1',	'Ldhb',	'Rab11a',	'Jup',	'4833423E24Rik',	'Prdx5',	'Krt23',	'Atp1b1',	'Cdkn1a',	'Serpinb2',	'Fcgbp',	'Rbm47',	'Sdc1',	'Trim29',	'Gsta2',	'Dgat2']
tuft_sign = ['Mctp1',	'Gng13',	'Avil',	'Rgs13',	'Espn',	'Cystm1',	'Anxa4',	'Lrmp',	'Ptpn18',	'Ivns1abp',	'Bmx',	'Fyb',	'Calm2',	'Ly6g6f',	'Ethe1',	'Reep5',	'1810046K07Rik',	'Smim22',	'Ociad2',	'Ptpn6',	'Abhd2',	'Cd24a',	'Ehf',	'Pde4d',	'Chil1',	'St18',	'Dgki',	'Pou2f3',	'Pik3r5',	'Trpm5',	'Lima1',	'Vav1',	'Alox5ap',	'Alox5',	'Krt8',	'Scand1',	'Tnc',	'Cox8a',	'Oxr1',	'Krt18',	'H2afj',	'Rab25',	'Gpcpd1',	'Strip2',	'Cox17',	'Sh2d6',	'Rassf6',	'Macrod2',	'Cldn7',	'Plk2']
nTEC_sign = ['Ptprn2',	'Cacna2d1',	'Scg5',	'Syt1',	'Krt8',	'Car8',	'Stxbp5l',	'Chga',	'Ccser1',	'Snap25',	'Cystm1',	'Krt7',	'Smim22',	'Cd9',	'Alcam',	'Krt18',	'Dnajc12',	'Pip5k1b',	'Epcam',	'Cldn7',	'Syt7',	'Kcnb2',	'Insm1',	'Cd24a',	'Cdh1',	'Cadps',	'Ceacam10',	'Resp18',	'Tsc22d1',	'Rims2',	'Cacna1a',	'Pcbd1',	'Cacnb2',	'Cplx2',	'Fam183b',	'Emb',	'AC149090.1',	'Rap1gap2',	'Aopep',	'Nol4',	'Stard10',	'Tmem163',	'5330417C22Rik',	'Pam',	'Btg2',	'Fhl2',	'Spint2',	'Actg1',	'A230057D06Rik',	'Guk1']
goblet_sign = ['Wfdc2',	'Fxyd3',	'H2-K1',	'Cyp2f2',	'Spint2',	'Atp1b1',	'S100a11',	'Gsto1',	'Cxcl17',	'Alcam',	'Krt18',	'Ly6e',	'Ly6d',	'Epcam',	'Ifi27l2a',	'Cbr2',	'Sorbs2',	'Smim22',	'Pglyrp1',	'H2-Q7',	'Serpinb11',	'Krt8',	'Krt19',	'Vamp8',	'Slc12a2',	'Cd24a',	'Irf7',	'Cldn3',	'Slc16a11',	'Gsta4',	'Cldn7',	'B2m',	'Elf3',	'Oasl2',	'Tst',	'Atp1a1',	'Aqp5',	'Sfn',	'Cdh1',	'Ezr',	'Tspan8',	'Tacstd2',	'Perp',	'Runx1',	'Eya2',	'Lmo7',	'Cd74',	'Rbm47',	'Chchd10',	'Bace2']
Mlike_sign = ['Ccl20',	'Ccl9',	'Serpinb6a',	'Serpinb1a',	'Tmsb4x',	'2200002D01Rik',	'Nostrin',	'Ubd',	'Csn2',	'Ctsh',	'Pold1',	'Bcl2a1d',	'Spib',	'Plb1',	'Fabp5',	'Marcksl1',	'Ccl6',	'Spint2',	'Cyp2a5',	'Clu',	'Bcl2a1b',	'Pglyrp1',	'Fabp1',	'Hamp',	'AW112010',	'Atox1',	'H2-M2',	'Atp6v1c1',	'Bcl2a1a',	'Krt20',	'Gjb2',	'Epcam',	'Iscu',	'Vamp8',	'Sephs2',	'4930520O04Rik',	'Mrpl38',	'Cldn7',	'Rac2',	'AA467197',	'Hspe1',	'Cib1',	'Dsg1a',	'Ahcyl2',	'Fxyd3',	'Tnfrsf11b',	'Srgn',	'Sept1',	'Fcgbp',	'Hist1h2bc']

In [ ]:
aaTEC1_sign = ['Zbtb20',	'Cd81',	'Trpm3',	'Slc16a11',	'Gstm1',	'Btg1',	'BC006965',	'Cldn3',	'S100a11',	'Gsta4',	'Ly6e',	'Cdk19',	'Wfdc18',	'Slc9a3r1',	'Ifi27l2a',	'Aldoc',	'Phlda1',	'Anxa2',	'Taldo1',	'Socs2',	'Fam107a',	'Krt18',	'Pbx1',	'Tm4sf1',	'Meis2',	'Gsn',	'Anxa1',	'Bmp6',	'Epcam',	'Dbi',	'Shank2',	'Kif19a',	'Isl1',	'Plin2',	'Ly6a',	'Nupr1',	'Id2',	'Rcn1',	'Gpc6',	'Rdh10',	'Cd24a',	'mt-Co1',	'Gab2',	'Pde4b',	'Slc5a8',	'Nedd4l',	'Fbxo2',	'Tshz2',	'Acss3',	'Cd9']
aaTEC2_sign = ['Csmd1',	'Zeb2',	'Mgp',	'Gsn',	'Zbtb20',	'Ndrg2',	'Serpine2',	'Cd81',	'Fxyd1',	'Laptm4a',	'Gpx3',	'Socs3',	'Igfbp7',	'Airn',	'Lgals1',	'Fos',	'Prrx1',	'Prkg1',	'Junb',	'Sparc',	'Cebpd',	'Apoe',	'Gstm1',	'Gadd45g',	'Id3',	'Egr1',	'Ptn',	'Cd63',	'Tm4sf1',	'Ccdc80',	'Phlda1',	'Cavin3',	'Cald1',	'Serping1',	'Sparcl1',	'Ccl19',	'Cst3',	'Cacna1c',	'Jund',	'Fstl1',	'Tagln2',	'Klf9',	'Jun',	'Gucy1a1',	'Col3a1',	'Notch3',	'Rora',	'Tshz2',	'Lhfp',	'Gnas']
cTEC_sign = ['Ctsl',	'Cstb',	'Cxcl12',	'Prss16',	'Krt18',	'Gas6',	'Pltp',	'Slc46a2',	'Ndufa11',	'Tbata',	'Ccl25',	'Psmb11',	'Tmem131l',	'Nlgn1',	'Prxl2b',	'Pax1',	'Wnt4',	'Dpp6',	'Psmb9',	'Bnip3l',	'Snhg11',	'Copz2',	'AI646519',	'Ank3',	'Lamp2',	'Ndrg3',	'Shisa2',	'Rgcc',	'Krt8',	'Rbfox1',	'Tsc22d1',	'Kctd1',	'Ly75',	'Sh2d4b',	'Igfbp5',	'Fabp5',	'Plgrkt',	'Spock2',	'Ryr3',	'Syngr1',	'Zfyve21',	'Tenm4',	'Trp63',	'Clic5',	'Gmpr',	'Castor1',	'Macf1',	'Ifi27',	'Limch1',	'Kcnk2']
mTEC1_sign = ['Apoe',	'Ifitm3',	'Rbms3',	'Ly6a',	'Ifi27l2a',	'Krt5',	'Krt14',	'Ctsl',	'Gas1',	'Ifitm2',	'Ccl21a',	'Itm2b',	'Gas6',	'Dcn',	'Isg15',	'Igfbp4',	'Laptm4a',	'Eya4',	'Cpne8',	'Mgp',	'Mir100hg',	'Sult5a1',	'Rtp4',	'Iigp1',	'Nxn',	'Rbp1',	'Gas5',	'Zfp36l1',	'Gsn',	'Ank3',	'Cst3',	'Pbx1',	'Lifr',	'Meis2',	'Perp',	'Oasl2',	'Anxa2',	'S100a10',	'Boc',	'Atp1a1',	'Ccl11',	'Phlda3',	'Ly6e',	'Nedd4',	'Myl6',	'Trp63',	'Anxa1',	'Tpt1',	'Hif1a',	'Gpx3']
mTECprol_sign = ['Tpm2',	'Krt17',	'Ascl1',	'Tmsb10',	'Ptma',	'Rgs5',	'Adm',	'H2afy2',	'Hes6',	'Krt7',	'Cdk4',	'Pfn1',	'Sox4',	'Skint10',	'Tubb6',	'Cfl1',	'Mapk13',	'Tubb2b',	'Wfdc18',	'Ccnd2',	'Stmn1',	'Mdk',	'Ccl19',	'Cd82',	'Gm49708',	'H3f3b',	'Lgals1',	'Prxl2b',	'Ybx1',	'Actg1',	'Eef1g',	'Nme2',	'Gm15943',	'Il11ra1',	'Colq',	'Cald1',	'Krt5',	'Hnrnpa1',	'Eif4a1',	'Ppia',	'Ccl21a',	'Tubb5',	'Hsp90ab1',	'Angptl2',	'Kcnq3',	'Lpo',	'Set',	'Hspa8',	'Hmgn1',	'Npm1']
mTEC2_sign = ['Srgn',	'Cyba',	'Ubd',	'Cd74',	'S100a14',	'Syt1',	'Cd52',	'Aire',	'H2-Aa',	'H2-Oa',	'Lrrc42',	'Mrpl38',	'H2-Eb1',	'Hagh',	'Nfkbia',	'Dpp10',	'Cdx1',	'Psme2',	'Hdc',	'Calcb',	'H2-DMb2',	'H2-Eb2',	'Fam89a',	'Gm47938',	'Hspb11',	'Fscn1',	'Csn2',	'Bspry',	'Ndufc2',	'Fezf2',	'Fabp5',	'Ankrd33b',	'Ing1',	'Gm48239',	'Utf1',	'Il4i1',	'Txn1',	'Ctss',	'Nup85',	'Cib1',	'Laptm5',	'Tm4sf5',	'Syngr2',	'Dock10',	'Dio1',	'Nsmce2',	'Snx29',	'S100g',	'Cd40',	'Ptprq']
mTEC3_sign = ['Ly6d',	'Dapl1',	'Dmkn',	'Spink5',	'Skint3',	'Fxyd3',	'Sfn',	'Tacstd2',	'Hspb1',	'Cdkn1a',	'Oit1',	'Pdzk1ip1',	'Prdx5',	'Cdkn2a',	'Perp',	'Cdkn2b',	'Gsta4',	'Atox1',	'Lypd3',	'Cst6',	'Metrnl',	'Sbsn',	'Dstn',	'Dsp',	'Calml3',	'Rab11a',	'Krt17',	'Krt23',	'Bcl2a1b',	'Serpinb2',	'H2afj',	'Jup',	'Avpi1',	'4833423E24Rik',	'Ide',	'Cebpb',	'Gsta2',	'Sdc1',	'Clic3',	'Tmem54',	'2200002D01Rik',	'Dgat2',	'Tmem45a',	'Cldn4',	'Rptn',	'Gltp',	'Nupr1',	'Arf6',	'Tspan8',	'Ier5']
tuft_sign = ['Mctp1',	'Gng13',	'Avil',	'Rgs13',	'Espn',	'Lrmp',	'Anxa4',	'Ltc4s',	'Ptpn18',	'Bmx',	'Cystm1',	'Ivns1abp',	'Fyb',	'Ly6g6f',	'Calm2',	'Ethe1',	'Reep5',	'Lima1',	'1810046K07Rik',	'Abhd2',	'Pik3r5',	'Trpm5',	'St18',	'Crip1',	'Dgki',	'Pou2f3',	'Vav1',	'Scand1',	'Chil1',	'Alox5ap',	'Ptpn6',	'Plac8',	'Alox5',	'Ostf1',	'Oxr1',	'Plk2',	'Ociad2',	'Sh2d6',	'Ahnak2',	'Stk38',	'Tmem245',	'Atp1a2',	'Aldh2',	'Cd47',	'Fxyd6',	'Inpp5d',	'Dclk1',	'Gpcpd1',	'Sh2d7',	'Strip2']
nTEC_sign = ['Ptprn2',	'Cacna2d1',	'Car8',	'Scg5',	'Stxbp5l',	'Snap25',	'Chga',	'Cd9',	'Dnajc12',	'Ccser1',	'Camk2n1',	'Syt7',	'Pam',	'Cacna1a',	'Syt1',	'Cacnb2',	'Cystm1',	'Tshz2',	'Fam183b',	'Cplx2',	'Ceacam10',	'Btg2',	'Rims2',	'Resp18',	'Fhl2',	'A230057D06Rik',	'Tmem163',	'Rap1gap2',	'Kcnb2',	'Nfasc',	'Pip5k1b',	'Alcam',	'Chgb',	'Ica1',	'Pcbd1',	'Krt7',	'Insm1',	'Smim22',	'Jund',	'5330417C22Rik',	'Atf3',	'AC149090.1',	'Bex2',	'Prkn',	'Cpn1',	'Tox3',	'Nol4',	'A330076H08Rik',	'Aopep',	'Stard10']
goblet_sign = ['Wfdc2',	'Cyp2f2',	'Cxcl17',	'Gsto1',	'Nupr1',	'S100a11',	'Anxa3',	'Sorbs2',	'H2-K1',	'Serpinb11',	'Krt19',	'Slc12a2',	'Fxyd3',	'Ly6d',	'Aqp5',	'Alcam',	'Bsg',	'Bace2',	'Ly6e',	'F3',	'Ly6a',	'Mecom',	'AW112010',	'Pglyrp1',	'Atp1b1',	'Cp',	'Anxa1',	'Tspan8',	'Irf7',	'Lgals3bp',	'Tceal9',	'Nfib',	'Gsta4',	'Cd14',	'Mllt3',	'Slc16a11',	'Upk1b',	'Lrrc26',	'Foxa1',	'Ece1',	'Timp2',	'Ifi27l2a',	'Ces1d',	'Epas1',	'Cbr2',	'Lmo7',	'Spink5',	'Hspb1',	'Smim22',	'Kcnj16']
Mlike_sign = ['Ccl20',	'Ccl9',	'Serpinb6a',	'Serpinb1a',	'Tmsb4x',	'2200002D01Rik',	'Nostrin',	'Ctsh',	'Ccl6',	'AW112010',	'Spib',	'Bcl2a1d',	'Csn2',	'Cyp2a5',	'Fabp1',	'Hamp',	'Pold1',	'Bcl2a1b',	'Atox1',	'Iscu',	'Pglyrp1',	'Bcl2a1a',	'Clu',	'Krt20',	'Atp6v1c1',	'H2-M2',	'Gjb2',	'Crip1',	'Fabp5',	'Ubd',	'Vamp5',	'Marcksl1',	'Plb1',	'Arpc1b',	'4930520O04Rik',	'Tnfrsf11b',	'Sephs2',	'Dsg1a',	'Spint2',	'Ftl1',	'AA467197',	'Sept1',	'Aif1',	'Gadd45a',	'Tnfaip2',	'Vamp8',	'Fabp4',	'Hspe1',	'Rac2',	'Ahcyl2']
capsFB_sign = ['Pi16',	'Timp2',	'Fn1',	'Opcml',	'Cd248',	'Mfap5',	'Anxa3',	'Pcolce2',	'Ackr3',	'Fndc1',	'Igfbp6',	'Sema3c',	'Clec3b',	'Creb5',	'Smpd3',	'Tmem100',	'Adgrd1',	'Ly6c1',	'Pcsk6',	'Nid1',	'Fbn1',	'Dpp4',	'Pla1a',	'Efhd1',	'Col14a1',	'Ebf2',	'Fstl1',	'Metrnl',	'Plpp3',	'Sdk1',	'Limch1',	'Axl',	'Loxl1',	'Anxa1',	'Emilin2',	'Tmsb4x',	'Tnxb',	'Gfpt2',	'Lsp1',	'Ly6a',	'Efna5',	'Adamts5',	'Igfbp5',	'Ugdh',	'Timp3',	'Ogn',	'Heg1',	'Islr',	'Ddr2',	'Ppp1r14b']
intFB_sign = ['Smoc2',	'Gpx3',	'Penk',	'Igf1',	'Gas1',	'Inmt',	'Pcolce',	'Lrp1',	'Itm2a',	'Gdf10',	'Rbp1',	'Fbln1',	'Sfrp1',	'Slit3',	'Svep1',	'Auts2',	'Lpl',	'Celf2',	'Serpinf1',	'Il11ra1',	'Adamts12',	'Abca8a',	'Cygb',	'Nfib',	'Dcn',	'Selenop',	'Ntrk2',	'Tmem119',	'Mgst1',	'Igfbp3',	'Mmp2',	'Ar',	'Gsn',	'Srpx',	'Col15a1',	'Mt1',	'Lum',	'Olfml3',	'Mfap2',	'Col3a1',	'Fxyd6',	'Fst',	'Cryab',	'Txnip',	'Htra3',	'Mfap4',	'Itm2b',	'Fbln2',	'Dhrs3',	'Col1a2']
medFB_sign = ['Csmd1',	'Serpine2',	'Enpp2',	'Ptn',	'Tmem176b',	'Tmem176a',	'B2m',	'Ccl19',	'Igfbp7',	'Apod',	'Bgn',	'H2-D1',	'Des',	'Ltc4s',	'Lhfp',	'Ifitm1',	'Cd9',	'Apoe',	'Lsamp',	'Ly6e',	'H2-K1',	'H2-Q7',	'Sparcl1',	'Il34',	'Ndufa4l2',	'Tcf4',	'Spon1',	'Colec12',	'Nrp1',	'Mylk',	'Hsd11b1',	'Mfge8',	'Jun',	'Cp',	'Meox1',	'Ecscr',	'Trps1',	'Ank2',	'Gpm6b',	'Cstb',	'Cd63',	'Pde4b',	'Postn',	'H3f3b',	'Mgp',	'Pten',	'Ptma',	'Ltbp1',	'Col15a1',	'Pde7b']
artEC_sign = ['Fbln5',	'Stmn2',	'Clu',	'Sema3g',	'Glul',	'Tm4sf1',	'Eps8l2',	'Col8a1',	'Nebl',	'Vegfc',	'Alpl',	'Arl15',	'Gja4',	'Edn1',	'S100a6',	'Mast4',	'Sox17',	'Pdgfd',	'Fbln2',	'Vim',	'Ltbp4',	'Slc6a6',	'Klf2',	'Heg1',	'Epas1',	'Fxyd5',	'Ly6a',	'Crip1',	'Fn1',	'Cd9',	'Mecom',	'Tsc22d1',	'Bsg',	'Atox1',	'Podxl',	'Ptprr',	'Ebf1',	'Cdk19',	'Icam2',	'Tspo',	'Cst3',	'Eln',	'Azin1',	'Pcsk5',	'Gadd45g',	'Timp3',	'Ace',	'Tanc2',	'Rgs10',	'Gja5']
capEC_sign = ['Gpihbp1',	'Rgcc',	'Fabp4',	'Cd36',	'Car4',	'Mgll',	'Cd300lg',	'Tcf15',	'Kdr',	'Arhgap18',	'Igfbp7',	'Kank3',	'Aqp7',	'Etl4',	'Tspan13',	'Cavin2',	'Hspb1',	'Ppp1r2',	'Lims2',	'Fabp5',	'Nrp1',	'Timp4',	'Gng11',	'Sept4',	'Lpl',	'Gm12002',	'Ccdc85a',	'Plpp3',	'Ablim3',	'Sparc',	'Xdh',	'Tmsb4x',	'AW112010',	'Ctnnbip1',	'Thrsp',	'Adgrl4',	'Cav2',	'Cxcl12',	'Dhrs3',	'Cd81',	'Emcn',	'Tcim',	'C1qtnf9',	'Sparcl1',	'Ccdc85b',	'Pitpnc1',	'Rflnb',	'Ubb',	'Ly6c1',	'Sult1a1']
venEC_sign = ['Lrg1',	'Il6st',	'Vwf',	'Plvap',	'mt-Co1',	'Selp',	'Spint2',	'Pcdh7',	'mt-Nd4',	'2200002D01Rik',	'Tmsb10',	'Eef1a1',	'mt-Co3',	'Bgn',	'Pde4d',	'mt-Atp6',	'Aqp1',	'Tpt1',	'Ctla2a',	'Ackr1',	'Apoe',	'Pdlim1',	'Cd9',	'St3gal4',	'Ehd4',	'Fth1',	'Eef1b2',	'Pecam1',	'Thsd7a',	'Col15a1',	'Man1a',	'Vim',	'Enpp2',	'Rbp1',	'Igfbp4',	'Abca1',	'Csrp2',	'Slco2b1',	'Zfp521',	'Vcam1',	'Dpysl3',	'Nr2f2',	'Pam',	'Ldb2',	'Insr',	'Cd74',	'Tmem176b',	'Il1r1',	'Arrb1',	'Arhgap26']

In [ ]:
x=9
sc.tl.score_genes(adata_transfer, gene_list = aaTEC1_sign[0:x],  score_name='aaTEC1_sign')
sc.tl.score_genes(adata_transfer, gene_list = aaTEC2_sign[0:x],  score_name='aaTEC2_sign')
sc.tl.score_genes(adata_transfer, gene_list = cTEC_sign[0:x],  score_name='cTEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC1_sign[0:x],  score_name='mTEC1_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTECprol_sign[0:x],  score_name='mTECprol_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC2_sign[0:x],  score_name='mTEC2_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC3_sign[0:x],  score_name='mTEC3_sign')
sc.tl.score_genes(adata_transfer, gene_list = tuft_sign[0:x],  score_name='tuft_sign')
sc.tl.score_genes(adata_transfer, gene_list = nTEC_sign[0:x],  score_name='nTEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = goblet_sign[0:x],  score_name='goblet_sign')
sc.tl.score_genes(adata_transfer, gene_list = Mlike_sign[0:x],  score_name='Mlike_sign')
sc.tl.score_genes(adata_transfer, gene_list = capsFB_sign[0:x],  score_name='capsFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = intFB_sign[0:x],  score_name='intFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = medFB_sign[0:x],  score_name='medFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = artEC_sign[0:x],  score_name='artEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = capEC_sign[0:x],  score_name='capEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = venEC_sign[0:x],  score_name='venEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = MEC_sign[0:x],  score_name='MEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = PCvSMC_sign[0:x],  score_name='PCvSMC_sign')
sc.tl.score_genes(adata_transfer, gene_list = nmSC_sign[0:x],  score_name='nmSC_sign')
sc.tl.score_genes(adata_transfer, gene_list = fat_sign[0:x],  score_name='fat_sign')

In [ ]:
x=9
sc.tl.score_genes(adata_transfer, gene_list = aaTEC1_sign[0:x],  score_name='aaTEC1_sign')
sc.tl.score_genes(adata_transfer, gene_list = aaTEC2_sign[0:x],  score_name='aaTEC2_sign')
sc.tl.score_genes(adata_transfer, gene_list = cTEC_sign[0:x],  score_name='cTEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC1_sign[0:x],  score_name='mTEC1_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTECprol_sign[0:x],  score_name='mTECprol_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC2_sign[0:x],  score_name='mTEC2_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC3_sign[0:x],  score_name='mTEC3_sign')
sc.tl.score_genes(adata_transfer, gene_list = tuft_sign[0:x],  score_name='tuft_sign')
sc.tl.score_genes(adata_transfer, gene_list = nTEC_sign[0:x],  score_name='nTEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = goblet_sign[0:x],  score_name='goblet_sign')
sc.tl.score_genes(adata_transfer, gene_list = Mlike_sign[0:x],  score_name='Mlike_sign')
sc.tl.score_genes(adata_transfer, gene_list = capsFB_sign[0:x],  score_name='capsFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = intFB_sign[0:x],  score_name='intFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = medFB_sign[0:x],  score_name='medFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = artEC_sign[0:x],  score_name='artEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = capEC_sign[0:x],  score_name='capEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = venEC_sign[0:x],  score_name='venEC_sign')

In [ ]:
aaTEC1_sign = ['Trpm3',	'Wfdc18','Cldn3',	'Slc16a11',	'Slc9a3r1',	'Btg1',	'Isl1',	'Pbx1',	'Cdk19',	'Ptprd',	'Taldo1','Aldoc',	'Socs2',	'Fam107a','Fbxo2',	'Nedd4l',	'Shank2','Kif19a',	'Bmp6',	'Id2',	'Arl4c',	'Slc5a8',	'Skp1a',	'Kirrel3',		'Prxl2a',	'Thsd4',	'Rdh10',	'Rhov',	'Plet1',	'Cited4',	'Galm',	'Rtl4',	'Gna14',	'Dclk2',	'Nipal2']
sc.tl.score_genes(adata_transfer, gene_list = aaTEC1_sign,  score_name='aaTEC1_sign')
aaTEC2_sign = ['Csmd1', 'Trpm3', 'Ndrg2', 'Spp1', 'Zeb2', 'Socs2', 'Kirrel3', 'Heyl', 'Bcl2', 'Olfm2', 'Ar', 'Cul1', 'Esr1' ]
sc.tl.score_genes(adata_transfer, gene_list = aaTEC2_sign,  score_name='aaTEC2_sign')
aaTEC1_sign_cherrypicked = ['Trpm3','Cldn3',	'Slc16a11','Isl1',	'Cdk19',	'Ptprd',	'Taldo1','Aldoc',	'Socs2',	'Fam107a','Fbxo2',	'Shank2','Kif19a',	'Bmp6', 'Slc5a8']


In [ ]:
aaTEC1_sign = ['Trpm3',	'Wfdc18','Cldn3',	'Slc16a11',	'Slc9a3r1',	'Btg1',	'Isl1',	'Pbx1',	'Cdk19',	'Ptprd',	'Taldo1','Aldoc',	'Socs2',	'Fam107a','Fbxo2',	'Nedd4l',	'Shank2','Kif19a',	'Bmp6',	'Id2',	'Arl4c',	'Slc5a8',	'Skp1a',	'Kirrel3',		'Prxl2a',	'Thsd4',	'Rdh10',	'Rhov',	'Plet1',	'Cited4',	'Galm',	'Rtl4',	'Gna14',	'Dclk2',	'Nipal2']
sc.tl.score_genes(adata_transfer_d0_18mo, gene_list = aaTEC1_sign,  score_name='aaTEC1_sign')
aaTEC2_sign = ['Csmd1', 'Trpm3', 'Ndrg2', 'Spp1', 'Zeb2', 'Socs2', 'Kirrel3', 'Heyl', 'Bcl2', 'Olfm2', 'Ar', 'Cul1', 'Esr1' ]
sc.tl.score_genes(adata_transfer, gene_list = aaTEC2_sign,  score_name='aaTEC2_sign')
aaTEC1_sign_cherrypicked = ['Trpm3','Cldn3',	'Slc16a11','Isl1',	'Cdk19',	'Ptprd',	'Taldo1','Aldoc',	'Socs2',	'Fam107a','Fbxo2',	'Shank2','Kif19a',	'Bmp6', 'Slc5a8']


In [ ]:
aaTEC1_sign = ['Trpm3',	'Wfdc18', 'Cldn3', 'Trpm3', 'Aldoc', 'Plin2', 'Fbxo2', 'Shank2', 'Kif19a', 'Bmp6']
sc.tl.score_genes(adata_transfer, gene_list = aaTEC1_sign,  score_name='aaTEC1_sign')

aaTEC1_sign = ['Trpm3',	'Wfdc18', 'Cldn3', 'Trpm3', 'Aldoc', 'Plin2', 'Fbxo2', 'Shank2', 'Kif19a', 'Bmp6']
sc.tl.score_genes(adata_transfer_d0_18mo, gene_list = aaTEC1_sign,  score_name='aaTEC1_sign')


In [ ]:
aaTEC2_sign = [ 'Csmd1', 'Trpm3', 'Ndrg2', 'Spp1', 'Zeb2', 'Socs2',  'Heyl']
sc.tl.score_genes(adata_transfer, gene_list = aaTEC2_sign,  score_name='aaTEC2_sign')

aaTEC2_sign = ['Csmd1', 'Trpm3', 'Ndrg2', 'Spp1', 'Zeb2', 'Socs2',  'Heyl']
sc.tl.score_genes(adata_transfer_d0_18mo, gene_list = aaTEC2_sign,  score_name='aaTEC2_sign')


In [ ]:
adata_transfer_d1_18mo = adata_transfer

In [ ]:
adata_transfer_d4_18mo = adata_transfer

In [ ]:
adata_transfer_d7_18mo = adata_transfer

In [ ]:
# day 0

sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(adata_transfer,
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'mTEC3_sign', 'medFB_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                  vmax=1.2,
                  vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
# day 1

sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(adata_transfer_d1_18mo,
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'mTEC3_sign', 'medFB_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                 # vmax=1.2,
                 # vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
# day 4

sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(adata_transfer_d4_18mo,
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'mTEC3_sign', 'medFB_sign','Lck'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                 # vmax=1.2,
                 # vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
# day 7

sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

sc.pl.spatial(adata_transfer_d7_18mo ,
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'mTEC3_sign', 'medFB_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                  vmax=1.2,
                  vmin=0.4
                  #save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_016"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'Thy1', 'Lck', 'Cd4'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                  vmax=1,
                  vmin=0.4,
                  save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_019"]:
    sc.pl.spatial(adata_transfer_d0_18mo[adata_transfer_d0_18mo.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'Thy1', 'Lck', 'Cd4'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                  vmax=1,
                  vmin=0.4,
                  save='_visium_18mo_d0_vmin-vmax.pdf'
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_019"]:
    sc.pl.spatial(adata_transfer_d0_18mo[adata_transfer_d0_18mo.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'Thy1', 'Lck', 'Cd4'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                  save='_visium_18mo_d0.pdf'
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'Thy1', 'Lck', 'Cd4'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6,
                  vmax=1,
                  vmin=0.4,
                  save='_visium_02mo_d0_vmin-vmax.pdf'

                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", "LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019", "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'mTEC1_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r',
                  ncols=7,
                  vmin=0
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", "LJ184_S_019"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', '10:aaTEC1', '11:aaTEC2', '12:cTEC', '13:mTEC1', '5:medFB'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", "LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019", "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['10:aaTEC1', '11:aaTEC2', '12:cTEC', '13:mTEC1', '14:mTEC-prol', '15:mTEC2', '16:mTEC3'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=4
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC2_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r'
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'cTEC_sign', 'mTEC1_sign', 'mTECprol_sign', 
                           'mTEC2_sign', 'mTEC3_sign', 'tuft_sign', 'nTEC_sign', 'goblet_sign', 'Mlike_sign',
                           'capsFB_sign', 'intFB_sign', 'medFB_sign', 'artEC_sign', 'capEC_sign', 'venEC_sign',
                           'MEC_sign', 'PCvSMC_sign', 'nmSC_sign', 'fat_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'cTEC_sign', 'mTEC1_sign', 'mTECprol_sign', 
                           'mTEC2_sign', 'mTEC3_sign', 'tuft_sign', 'nTEC_sign', 'goblet_sign', 'Mlike_sign',
                           'capsFB_sign', 'intFB_sign', 'medFB_sign', 'artEC_sign', 'capEC_sign', 'venEC_sign',
                           'MEC_sign', 'PCvSMC_sign', 'nmSC_sign', 'fat_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'cTEC_sign', 'mTEC1_sign', 'mTECprol_sign', 
                           'mTEC2_sign', 'mTEC3_sign', 'tuft_sign', 'nTEC_sign', 'goblet_sign', 'Mlike_sign',
                           'capsFB_sign', 'intFB_sign', 'medFB_sign', 'artEC_sign', 'capEC_sign', 'venEC_sign',
                           'MEC_sign', 'PCvSMC_sign', 'nmSC_sign', 'fat_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'cTEC_sign', 'mTEC1_sign', 'mTECprol_sign', 
                           'mTEC2_sign', 'mTEC3_sign', 'tuft_sign', 'nTEC_sign', 'goblet_sign', 'Mlike_sign',
                           'capsFB_sign', 'intFB_sign', 'medFB_sign', 'artEC_sign', 'capEC_sign', 'venEC_sign',
                           'MEC_sign', 'PCvSMC_sign', 'nmSC_sign', 'fat_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
### Within cell type signatures

In [ ]:
aaTEC1_sign = ['Zbtb20',	'Cd81',	'Trpm3',	'Slc16a11',	'Gstm1',	'Btg1',	'BC006965',	'Cldn3',	'S100a11',	'Gsta4',	'Ly6e',	'Cdk19',	'Wfdc18',	'Slc9a3r1',	'Ifi27l2a',	'Aldoc',	'Phlda1',	'Anxa2',	'Taldo1',	'Socs2',	'Fam107a',	'Krt18',	'Pbx1',	'Tm4sf1',	'Meis2',	'Gsn',	'Anxa1',	'Bmp6',	'Epcam',	'Dbi',	'Shank2',	'Kif19a',	'Isl1',	'Plin2',	'Ly6a',	'Nupr1',	'Id2',	'Rcn1',	'Gpc6',	'Rdh10',	'Cd24a',	'mt-Co1',	'Gab2',	'Pde4b',	'Slc5a8',	'Nedd4l',	'Fbxo2',	'Tshz2',	'Acss3',	'Cd9']
aaTEC2_sign = ['Csmd1',	'Zeb2',	'Mgp',	'Gsn',	'Zbtb20',	'Ndrg2',	'Serpine2',	'Cd81',	'Fxyd1',	'Laptm4a',	'Gpx3',	'Socs3',	'Igfbp7',	'Airn',	'Lgals1',	'Fos',	'Prrx1',	'Prkg1',	'Junb',	'Sparc',	'Cebpd',	'Apoe',	'Gstm1',	'Gadd45g',	'Id3',	'Egr1',	'Ptn',	'Cd63',	'Tm4sf1',	'Ccdc80',	'Phlda1',	'Cavin3',	'Cald1',	'Serping1',	'Sparcl1',	'Ccl19',	'Cst3',	'Cacna1c',	'Jund',	'Fstl1',	'Tagln2',	'Klf9',	'Jun',	'Gucy1a1',	'Col3a1',	'Notch3',	'Rora',	'Tshz2',	'Lhfp',	'Gnas']
cTEC_sign = ['Ctsl',	'Cstb',	'Cxcl12',	'Prss16',	'Krt18',	'Gas6',	'Pltp',	'Slc46a2',	'Ndufa11',	'Tbata',	'Ccl25',	'Psmb11',	'Tmem131l',	'Nlgn1',	'Prxl2b',	'Pax1',	'Wnt4',	'Dpp6',	'Psmb9',	'Bnip3l',	'Snhg11',	'Copz2',	'AI646519',	'Ank3',	'Lamp2',	'Ndrg3',	'Shisa2',	'Rgcc',	'Krt8',	'Rbfox1',	'Tsc22d1',	'Kctd1',	'Ly75',	'Sh2d4b',	'Igfbp5',	'Fabp5',	'Plgrkt',	'Spock2',	'Ryr3',	'Syngr1',	'Zfyve21',	'Tenm4',	'Trp63',	'Clic5',	'Gmpr',	'Castor1',	'Macf1',	'Ifi27',	'Limch1',	'Kcnk2']
mTEC1_sign = ['Apoe',	'Ifitm3',	'Rbms3',	'Ly6a',	'Ifi27l2a',	'Krt5',	'Krt14',	'Ctsl',	'Gas1',	'Ifitm2',	'Ccl21a',	'Itm2b',	'Gas6',	'Dcn',	'Isg15',	'Igfbp4',	'Laptm4a',	'Eya4',	'Cpne8',	'Mgp',	'Mir100hg',	'Sult5a1',	'Rtp4',	'Iigp1',	'Nxn',	'Rbp1',	'Gas5',	'Zfp36l1',	'Gsn',	'Ank3',	'Cst3',	'Pbx1',	'Lifr',	'Meis2',	'Perp',	'Oasl2',	'Anxa2',	'S100a10',	'Boc',	'Atp1a1',	'Ccl11',	'Phlda3',	'Ly6e',	'Nedd4',	'Myl6',	'Trp63',	'Anxa1',	'Tpt1',	'Hif1a',	'Gpx3']
mTECprol_sign = ['Tpm2',	'Krt17',	'Ascl1',	'Tmsb10',	'Ptma',	'Rgs5',	'Adm',	'H2afy2',	'Hes6',	'Krt7',	'Cdk4',	'Pfn1',	'Sox4',	'Skint10',	'Tubb6',	'Cfl1',	'Mapk13',	'Tubb2b',	'Wfdc18',	'Ccnd2',	'Stmn1',	'Mdk',	'Ccl19',	'Cd82',	'Gm49708',	'H3f3b',	'Lgals1',	'Prxl2b',	'Ybx1',	'Actg1',	'Eef1g',	'Nme2',	'Gm15943',	'Il11ra1',	'Colq',	'Cald1',	'Krt5',	'Hnrnpa1',	'Eif4a1',	'Ppia',	'Ccl21a',	'Tubb5',	'Hsp90ab1',	'Angptl2',	'Kcnq3',	'Lpo',	'Set',	'Hspa8',	'Hmgn1',	'Npm1']
mTEC2_sign = ['Srgn',	'Cyba',	'Ubd',	'Cd74',	'S100a14',	'Syt1',	'Cd52',	'Aire',	'H2-Aa',	'H2-Oa',	'Lrrc42',	'Mrpl38',	'H2-Eb1',	'Hagh',	'Nfkbia',	'Dpp10',	'Cdx1',	'Psme2',	'Hdc',	'Calcb',	'H2-DMb2',	'H2-Eb2',	'Fam89a',	'Gm47938',	'Hspb11',	'Fscn1',	'Csn2',	'Bspry',	'Ndufc2',	'Fezf2',	'Fabp5',	'Ankrd33b',	'Ing1',	'Gm48239',	'Utf1',	'Il4i1',	'Txn1',	'Ctss',	'Nup85',	'Cib1',	'Laptm5',	'Tm4sf5',	'Syngr2',	'Dock10',	'Dio1',	'Nsmce2',	'Snx29',	'S100g',	'Cd40',	'Ptprq']
mTEC3_sign = ['Ly6d',	'Dapl1',	'Dmkn',	'Spink5',	'Skint3',	'Fxyd3',	'Sfn',	'Tacstd2',	'Hspb1',	'Cdkn1a',	'Oit1',	'Pdzk1ip1',	'Prdx5',	'Cdkn2a',	'Perp',	'Cdkn2b',	'Gsta4',	'Atox1',	'Lypd3',	'Cst6',	'Metrnl',	'Sbsn',	'Dstn',	'Dsp',	'Calml3',	'Rab11a',	'Krt17',	'Krt23',	'Bcl2a1b',	'Serpinb2',	'H2afj',	'Jup',	'Avpi1',	'4833423E24Rik',	'Ide',	'Cebpb',	'Gsta2',	'Sdc1',	'Clic3',	'Tmem54',	'2200002D01Rik',	'Dgat2',	'Tmem45a',	'Cldn4',	'Rptn',	'Gltp',	'Nupr1',	'Arf6',	'Tspan8',	'Ier5']
tuft_sign = ['Mctp1',	'Gng13',	'Avil',	'Rgs13',	'Espn',	'Lrmp',	'Anxa4',	'Ltc4s',	'Ptpn18',	'Bmx',	'Cystm1',	'Ivns1abp',	'Fyb',	'Ly6g6f',	'Calm2',	'Ethe1',	'Reep5',	'Lima1',	'1810046K07Rik',	'Abhd2',	'Pik3r5',	'Trpm5',	'St18',	'Crip1',	'Dgki',	'Pou2f3',	'Vav1',	'Scand1',	'Chil1',	'Alox5ap',	'Ptpn6',	'Plac8',	'Alox5',	'Ostf1',	'Oxr1',	'Plk2',	'Ociad2',	'Sh2d6',	'Ahnak2',	'Stk38',	'Tmem245',	'Atp1a2',	'Aldh2',	'Cd47',	'Fxyd6',	'Inpp5d',	'Dclk1',	'Gpcpd1',	'Sh2d7',	'Strip2']
nTEC_sign = ['Ptprn2',	'Cacna2d1',	'Car8',	'Scg5',	'Stxbp5l',	'Snap25',	'Chga',	'Cd9',	'Dnajc12',	'Ccser1',	'Camk2n1',	'Syt7',	'Pam',	'Cacna1a',	'Syt1',	'Cacnb2',	'Cystm1',	'Tshz2',	'Fam183b',	'Cplx2',	'Ceacam10',	'Btg2',	'Rims2',	'Resp18',	'Fhl2',	'A230057D06Rik',	'Tmem163',	'Rap1gap2',	'Kcnb2',	'Nfasc',	'Pip5k1b',	'Alcam',	'Chgb',	'Ica1',	'Pcbd1',	'Krt7',	'Insm1',	'Smim22',	'Jund',	'5330417C22Rik',	'Atf3',	'AC149090.1',	'Bex2',	'Prkn',	'Cpn1',	'Tox3',	'Nol4',	'A330076H08Rik',	'Aopep',	'Stard10']
goblet_sign = ['Wfdc2',	'Cyp2f2',	'Cxcl17',	'Gsto1',	'Nupr1',	'S100a11',	'Anxa3',	'Sorbs2',	'H2-K1',	'Serpinb11',	'Krt19',	'Slc12a2',	'Fxyd3',	'Ly6d',	'Aqp5',	'Alcam',	'Bsg',	'Bace2',	'Ly6e',	'F3',	'Ly6a',	'Mecom',	'AW112010',	'Pglyrp1',	'Atp1b1',	'Cp',	'Anxa1',	'Tspan8',	'Irf7',	'Lgals3bp',	'Tceal9',	'Nfib',	'Gsta4',	'Cd14',	'Mllt3',	'Slc16a11',	'Upk1b',	'Lrrc26',	'Foxa1',	'Ece1',	'Timp2',	'Ifi27l2a',	'Ces1d',	'Epas1',	'Cbr2',	'Lmo7',	'Spink5',	'Hspb1',	'Smim22',	'Kcnj16']
Mlike_sign = ['Ccl20',	'Ccl9',	'Serpinb6a',	'Serpinb1a',	'Tmsb4x',	'2200002D01Rik',	'Nostrin',	'Ctsh',	'Ccl6',	'AW112010',	'Spib',	'Bcl2a1d',	'Csn2',	'Cyp2a5',	'Fabp1',	'Hamp',	'Pold1',	'Bcl2a1b',	'Atox1',	'Iscu',	'Pglyrp1',	'Bcl2a1a',	'Clu',	'Krt20',	'Atp6v1c1',	'H2-M2',	'Gjb2',	'Crip1',	'Fabp5',	'Ubd',	'Vamp5',	'Marcksl1',	'Plb1',	'Arpc1b',	'4930520O04Rik',	'Tnfrsf11b',	'Sephs2',	'Dsg1a',	'Spint2',	'Ftl1',	'AA467197',	'Sept1',	'Aif1',	'Gadd45a',	'Tnfaip2',	'Vamp8',	'Fabp4',	'Hspe1',	'Rac2',	'Ahcyl2']
capsFB_sign = ['Pi16',	'Timp2',	'Fn1',	'Opcml',	'Cd248',	'Mfap5',	'Anxa3',	'Pcolce2',	'Ackr3',	'Fndc1',	'Igfbp6',	'Sema3c',	'Clec3b',	'Creb5',	'Smpd3',	'Tmem100',	'Adgrd1',	'Ly6c1',	'Pcsk6',	'Nid1',	'Fbn1',	'Dpp4',	'Pla1a',	'Efhd1',	'Col14a1',	'Ebf2',	'Fstl1',	'Metrnl',	'Plpp3',	'Sdk1',	'Limch1',	'Axl',	'Loxl1',	'Anxa1',	'Emilin2',	'Tmsb4x',	'Tnxb',	'Gfpt2',	'Lsp1',	'Ly6a',	'Efna5',	'Adamts5',	'Igfbp5',	'Ugdh',	'Timp3',	'Ogn',	'Heg1',	'Islr',	'Ddr2',	'Ppp1r14b']
intFB_sign = ['Smoc2',	'Gpx3',	'Penk',	'Igf1',	'Gas1',	'Inmt',	'Pcolce',	'Lrp1',	'Itm2a',	'Gdf10',	'Rbp1',	'Fbln1',	'Sfrp1',	'Slit3',	'Svep1',	'Auts2',	'Lpl',	'Celf2',	'Serpinf1',	'Il11ra1',	'Adamts12',	'Abca8a',	'Cygb',	'Nfib',	'Dcn',	'Selenop',	'Ntrk2',	'Tmem119',	'Mgst1',	'Igfbp3',	'Mmp2',	'Ar',	'Gsn',	'Srpx',	'Col15a1',	'Mt1',	'Lum',	'Olfml3',	'Mfap2',	'Col3a1',	'Fxyd6',	'Fst',	'Cryab',	'Txnip',	'Htra3',	'Mfap4',	'Itm2b',	'Fbln2',	'Dhrs3',	'Col1a2']
medFB_sign = ['Csmd1',	'Serpine2',	'Enpp2',	'Ptn',	'Tmem176b',	'Tmem176a',	'B2m',	'Ccl19',	'Igfbp7',	'Apod',	'Bgn',	'H2-D1',	'Des',	'Ltc4s',	'Lhfp',	'Ifitm1',	'Cd9',	'Apoe',	'Lsamp',	'Ly6e',	'H2-K1',	'H2-Q7',	'Sparcl1',	'Il34',	'Ndufa4l2',	'Tcf4',	'Spon1',	'Colec12',	'Nrp1',	'Mylk',	'Hsd11b1',	'Mfge8',	'Jun',	'Cp',	'Meox1',	'Ecscr',	'Trps1',	'Ank2',	'Gpm6b',	'Cstb',	'Cd63',	'Pde4b',	'Postn',	'H3f3b',	'Mgp',	'Pten',	'Ptma',	'Ltbp1',	'Col15a1',	'Pde7b']
artEC_sign = ['Fbln5',	'Stmn2',	'Clu',	'Sema3g',	'Glul',	'Tm4sf1',	'Eps8l2',	'Col8a1',	'Nebl',	'Vegfc',	'Alpl',	'Arl15',	'Gja4',	'Edn1',	'S100a6',	'Mast4',	'Sox17',	'Pdgfd',	'Fbln2',	'Vim',	'Ltbp4',	'Slc6a6',	'Klf2',	'Heg1',	'Epas1',	'Fxyd5',	'Ly6a',	'Crip1',	'Fn1',	'Cd9',	'Mecom',	'Tsc22d1',	'Bsg',	'Atox1',	'Podxl',	'Ptprr',	'Ebf1',	'Cdk19',	'Icam2',	'Tspo',	'Cst3',	'Eln',	'Azin1',	'Pcsk5',	'Gadd45g',	'Timp3',	'Ace',	'Tanc2',	'Rgs10',	'Gja5']
capEC_sign = ['Gpihbp1',	'Rgcc',	'Fabp4',	'Cd36',	'Car4',	'Mgll',	'Cd300lg',	'Tcf15',	'Kdr',	'Arhgap18',	'Igfbp7',	'Kank3',	'Aqp7',	'Etl4',	'Tspan13',	'Cavin2',	'Hspb1',	'Ppp1r2',	'Lims2',	'Fabp5',	'Nrp1',	'Timp4',	'Gng11',	'Sept4',	'Lpl',	'Gm12002',	'Ccdc85a',	'Plpp3',	'Ablim3',	'Sparc',	'Xdh',	'Tmsb4x',	'AW112010',	'Ctnnbip1',	'Thrsp',	'Adgrl4',	'Cav2',	'Cxcl12',	'Dhrs3',	'Cd81',	'Emcn',	'Tcim',	'C1qtnf9',	'Sparcl1',	'Ccdc85b',	'Pitpnc1',	'Rflnb',	'Ubb',	'Ly6c1',	'Sult1a1']
venEC_sign = ['Lrg1',	'Il6st',	'Vwf',	'Plvap',	'mt-Co1',	'Selp',	'Spint2',	'Pcdh7',	'mt-Nd4',	'2200002D01Rik',	'Tmsb10',	'Eef1a1',	'mt-Co3',	'Bgn',	'Pde4d',	'mt-Atp6',	'Aqp1',	'Tpt1',	'Ctla2a',	'Ackr1',	'Apoe',	'Pdlim1',	'Cd9',	'St3gal4',	'Ehd4',	'Fth1',	'Eef1b2',	'Pecam1',	'Thsd7a',	'Col15a1',	'Man1a',	'Vim',	'Enpp2',	'Rbp1',	'Igfbp4',	'Abca1',	'Csrp2',	'Slco2b1',	'Zfp521',	'Vcam1',	'Dpysl3',	'Nr2f2',	'Pam',	'Ldb2',	'Insr',	'Cd74',	'Tmem176b',	'Il1r1',	'Arrb1',	'Arhgap26']

In [ ]:
x=19
sc.tl.score_genes(adata_transfer, gene_list = aaTEC1_sign[0:x],  score_name='aaTEC1_sign')
sc.tl.score_genes(adata_transfer, gene_list = aaTEC2_sign[0:x],  score_name='aaTEC2_sign')
sc.tl.score_genes(adata_transfer, gene_list = cTEC_sign[0:x],  score_name='cTEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC1_sign[0:x],  score_name='mTEC1_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTECprol_sign[0:x],  score_name='mTECprol_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC2_sign[0:x],  score_name='mTEC2_sign')
sc.tl.score_genes(adata_transfer, gene_list = mTEC3_sign[0:x],  score_name='mTEC3_sign')
sc.tl.score_genes(adata_transfer, gene_list = tuft_sign[0:x],  score_name='tuft_sign')
sc.tl.score_genes(adata_transfer, gene_list = nTEC_sign[0:x],  score_name='nTEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = goblet_sign[0:x],  score_name='goblet_sign')
sc.tl.score_genes(adata_transfer, gene_list = Mlike_sign[0:x],  score_name='Mlike_sign')
sc.tl.score_genes(adata_transfer, gene_list = capsFB_sign[0:x],  score_name='capsFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = intFB_sign[0:x],  score_name='intFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = medFB_sign[0:x],  score_name='medFB_sign')
sc.tl.score_genes(adata_transfer, gene_list = artEC_sign[0:x],  score_name='artEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = capEC_sign[0:x],  score_name='capEC_sign')
sc.tl.score_genes(adata_transfer, gene_list = venEC_sign[0:x],  score_name='venEC_sign')

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'Trpm3', 'Lck'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r',
                  ncols=5,
                  vmin=0.5,
                  vmax=3 
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC2_sign', 'Trpm3', 'Lck'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r',
                  ncols=5,
                  vmin=0,
                  vmax=1 
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'cTEC_sign', 'mTEC1_sign', 'mTECprol_sign', 
                           'mTEC2_sign', 'mTEC3_sign', 'tuft_sign', 'nTEC_sign', 'goblet_sign', 'Mlike_sign',
                           'capsFB_sign', 'intFB_sign', 'medFB_sign', 'artEC_sign', 'capEC_sign', 'venEC_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'cTEC_sign', 'mTEC1_sign', 'mTECprol_sign', 
                           'mTEC2_sign', 'mTEC3_sign', 'tuft_sign', 'nTEC_sign', 'goblet_sign', 'Mlike_sign',
                           'capsFB_sign', 'intFB_sign', 'medFB_sign', 'artEC_sign', 'capEC_sign', 'venEC_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
adata_transfer.uns['leiden_0.2_colors'] = ['#7fc97f', '#f0027f']
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", #"LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019"]:#, "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['leiden_0.2', 'aaTEC1_sign', 'aaTEC2_sign', 'cTEC_sign', 'mTEC1_sign', 'mTECprol_sign', 
                           'mTEC2_sign', 'mTEC3_sign', 'tuft_sign', 'nTEC_sign', 'goblet_sign', 'Mlike_sign',
                           'capsFB_sign', 'intFB_sign', 'medFB_sign', 'artEC_sign', 'capEC_sign', 'venEC_sign'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=6
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", "LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019", "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['10:aaTEC1', '11:aaTEC2', '12:cTEC', '13:mTEC1', '14:mTEC-prol', '15:mTEC2', '16:mTEC3'],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=4
                 )

In [ ]:
sc.set_figure_params(dpi=80, dpi_save=300, color_map='viridis', vector_friendly=True, transparent=True)

for library in ["LJ184_S_003", "LJ184_S_008", "LJ184_S_011", "LJ184_S_016", 
                "LJ184_S_019", "LJ184_S_021", "LJ184_S_023", "LJ184_S_026"]:
    sc.pl.spatial(adata_transfer[adata_transfer.obs.library_id == library,:], library_id=library, 
                  img_key="hires",
                  color = ['Thy1', 'Lck', 'Krt5', 'Krt14', 'Krt8', 'Krt10' ],
                  size=1.5,
                  use_raw=False,
                  color_map='Spectral_r', 
                  ncols=4
                 )

In [ ]:
aaTEC1_sign = ['Zbtb20',	'Cd81',	'Trpm3',	'Slc16a11',	'Gstm1',	'Btg1',	'BC006965',	'Cldn3',	'Gsta4',	'Cdk19',	'Slc9a3r1','Aldoc',	'Phlda1',	'Taldo1',	'Socs2',	'Fam107a',	'Tm4sf1',	'Bmp6',	'Shank2',	'Kif19a',	'Isl1',	'Plin2',	'Nupr1',	'Id2',	'Rcn1',	'Gpc6',	'Rdh10',	'Cd24a',	'Slc5a8',	'Nedd4l',	'Fbxo2',	'Acss3']

In [ ]:
aaTEC2_sign = ['Csmd1',	'Zeb2',	'Mgp',	'Gsn',	'Zbtb20',	'Ndrg2',	'Serpine2',	'Cd81',	'Fxyd1',	'Laptm4a',	'Gpx3',	'Socs3',	'Igfbp7',	'Airn',	'Lgals1',	'Fos',	'Prrx1',	'Prkg1',	'Junb',	'Sparc']